In [1]:
print('a')

a


In [2]:
'''
Convert synthesize data to 
entity2id.txt, relation2id, entity2id_objType.txt
'''

import json
import pickle
import os
from tqdm.notebook import tqdm
from typing import Set, Tuple, Generator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


OUTPUT_PATH = './data/3_openKE/synthesize'
INPUT_PATH = '/workdir/home/bai/data_processing/dgl/data_new/DARPA/before_embedding/3.10/synthesized_format.json'


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)


In [3]:
data = list()
with open(INPUT_PATH, 'r') as f:
    for line in f:
        event = json.loads(line)
        data.append(event)
        

# # Process each line as a separate JSON object
# for line in tqdm(data, desc='Loading JSON'):
#     try:
#         item = json.loads(line)  # Parse the JSON object
#         if item.get('label') == 'benign':
#             print(item)
#             break
            
#     except json.JSONDecodeError as e:
#         print(f"Error decoding JSON: {e}")

In [4]:
print(len(data))
type(data[1])

42899811


dict

In [5]:
data[1]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432},
 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting',
  'Type': 'file'},
 'relation': 'EVENT_OPEN',
 'label': 'benign'}

In [6]:
import json
import yaml

def read_yaml(file_path):
    """Read a YAML file and return the parsed data."""
    with open(file_path, 'r') as file:
        return yaml.safe_load(file)

yaml_file_path = './data/darpa_synthesized_mapping.yml'

yaml_file = read_yaml(yaml_file_path)

In [7]:
print(type(yaml_file))
print(len(yaml_file))
# print(yaml_file)
yaml_file

<class 'dict'>
4


{'file': {'attr': {'filepath': 'File Name'},
  'relation': {'EVENT_CREATE_OBJECT': 'CreateFile',
   'EVENT_CHANGE_PRINCIPAL': 'SetBasicInformationFile',
   'EVENT_EXECUTE': 'Process Create',
   'EVENT_LOADLIBRARY': 'Load Image',
   'EVENT_MODIFY_FILE_ATTRIBUTES': 'SetBasicInformationFile',
   'EVENT_OPEN': 'CreateFile',
   'EVENT_READ': 'ReadFile',
   'EVENT_UNLINK': 'SetDispositionInformationFile',
   'EVENT_WRITE': 'WriteFile',
   'EVENT_CHECK_FILE_ATTRIBUTES': 'QueryBasicInformationFile',
   'EVENT_CLOSE': 'CloseFile',
   'EVENT_RENAME': 'SetRenameInformationFile',
   'EVENT_FCNTL': 'CreateFileMapping',
   'EVENT_OTHER': 'NO_CORRESPONDING_RELATION'}},
 'network': {'attr': {'localAddress': 'Source Ip',
   'localPort': 'Source Port',
   'remoteAddress': 'Destination Ip',
   'remotePort': 'Destination Port'},
  'relation': {'EVENT_CONNECT': 'TCP Connect',
   'EVENT_RECVFROM': 'TCP Receive',
   'EVENT_SENDTO': 'TCP Send',
   'EVENT_ACCEPT': 'TCP Accept',
   'EVENT_BIND': 'NO_CORRESPONDI

In [8]:
def map_relation_to_synthesized(data, mapping):
    """Map relation in the data from DARPA format to synthesized format using the provided mapping."""
    for event in data:
        src_type = event['src_node']['Type']
        dst_type = event['dst_node']['Type']
        darpa_relation = event['relation']

        # Check if the relation mapping exists for source type
        if darpa_relation in mapping[src_type]['relation']:
            event['relation'] = mapping[src_type]['relation'][darpa_relation]
        # If not found in source type, check in destination type
        elif darpa_relation in mapping[dst_type]['relation']:
            event['relation'] = mapping[dst_type]['relation'][darpa_relation]
        # If not found in both, set to 'NO_CORRESPONDING_RELATION'
        else:
            event['relation'] = "NO_CORRESPONDING_RELATION"
    return data

mapped_data = map_relation_to_synthesized(data, yaml_file)
# data_test

In [9]:
mapped_data[1]

{'src_uuid': '966D983C-607E-4039-BEEF-5E76BC0A4DA8',
 'src_node': {'Cmdline': 'None', 'Type': 'process', 'Pid': 9432},
 'dst_uuid': '1987EE68-E918-4765-8082-3DE89890A305',
 'dst_node': {'Filepath': '\\Device\\HarddiskVolume2\\Users\\admin\\AppData\\Roaming\\Thunderbird\\Profiles\\isfi13ur.default\\datareporting',
  'Type': 'file'},
 'relation': 'Process Create',
 'label': 'benign'}

In [10]:
count = 0

for event in data_test:
    if event['relation'] == "NO_CORRESPONDING_RELATION":
        count += 1
        print(event)
        if count == 5:
            break

NameError: name 'data_test' is not defined

In [ ]:
def print_events_with_no_corresponding_relation(original_data, mapped_data):
    for original_event, mapped_event in zip(original_data, mapped_data):
        if mapped_event['relation'] == "NO_CORRESPONDING_RELATION":
            print("Original Event:", original_event)
            print("Mapped Event:", mapped_event)
            print()  # Newline for separation
        
        
print_events_with_no_corresponding_relation(original_data, mapped_data)

In [29]:
# keys = set()
# for d in tqdm(data):
#     if d['dst_node']['Type'] == 'network':
#         keys = keys.union(d['dst_node'].keys())
# keys 

keys = set()
for d in tqdm(data):
    if d['dst_node'].get('Type') == 'network':
        keys.update(d['dst_node'].keys())

print(keys)

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Localaddress', 'Remoteport', 'Ipprotocol', 'Localport', 'Type', 'Remoteaddress'}


In [40]:
# keys = set()
# for d in tqdm(data):
#     if d['dst_node']['Type'] == 'network':
#         keys = keys.union(d['dst_node'].keys())
# keys 

keys = set()
for d in tqdm(data):
    if d['dst_node'].get('Type') == 'network':
        keys.update(d['dst_node'].keys())

print(keys)

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Localaddress', 'Remoteport', 'Ipprotocol', 'Localport', 'Type', 'Remoteaddress'}


In [17]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'process':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Cmdline', 'Pid', 'Type'}

In [18]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'registry':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Key', 'Type'}

In [19]:
keys = set()
for d in tqdm(data):
    if d['dst_node']['Type'] == 'file':
        keys = keys.union(d['dst_node'].keys())
keys

  0%|          | 0/42899811 [00:00<?, ?it/s]

{'Filepath', 'Type'}

In [20]:
save_data = list()
for d in tqdm(data):
    src_uuid = d['src_uuid']
    src_node = d['src_node']
    dst_uuid = d['dst_uuid']
    dst_node = d['dst_node']
    relation = d['relation']
    label = d['label']
    
    src_cmdline = 'nan' if type(src_node['Cmdline']) == float and np.isnan(src_node['Cmdline']) else src_node['Cmdline']
    dst_value = 'nan' if 'Value' not in dst_node else dst_node['Value']
    
    new_row = {
        'src_uuid': src_uuid, 
#         'src': f'process:Cmdline: {cmdline}\nImage: {src_node["Image"]}\nName: {src_node["Name"]}\nPid: {src_node["Pid"]}',
#         'src': f'process:{src_node["Image"]}&{src_cmdline}&{src_node["Pid"]}',
        'src': f'process:{src_node["Pid"]}&{src_cmdline}',
        'dst_uuid': dst_uuid, 
        'dst': None, 
        'relation': relation, 
        'dst_type': dst_node['Type'].lower(),
        'label': label
    }

    
    if new_row['dst_type'] == 'file':
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node['Filepath']
    elif new_row['dst_type'] == 'network':
#         new_row['dst'] = new_row['dst_type'] + ':' + dst_node['Localport'] + '&' + dst_node['Localaddress'] + '&' + dst_node['Remoteaddress'] + '&'+ dst_node['Ipprotocol'] + '&'  + dst_node['Remoteport']
        new_row['dst'] = new_row['dst_type'] + ':' + str(dst_node['Localport']) + '&' + str(dst_node['Localaddress']) + '&' + str(dst_node['Remoteaddress']) + '&'+ str(dst_node['Ipprotocol']) + '&'  + str(dst_node['Remoteport'])

    elif new_row['dst_type'] == 'registry':
        new_row['dst'] = new_row['dst_type'] + ':' + dst_node['Key']
    elif new_row['dst_type'] == 'process':
#         new_row['dst'] = new_row['dst_type'] + ':' + f'{dst_node["Cmdline"]}&{dst_node["Image"]}&{dst_node["Name"]}&{dst_node["Pid"]}'
        new_row['dst'] = new_row['dst_type'] + ':' + f'{dst_node["Pid"]}&{dst_node["Cmdline"]}'
#         new_row['dst'] = new_row['dst_type'] + ':' + f'{dst_node["Image"]}&{dst_node["Cmdline"]}${dst_node["Pid"]}'
        
    new_row['src'] = new_row['src'].replace(' ', '_')
    new_row['dst'] = new_row['dst'].replace(' ', '_')
    
#     if new_row['dst'] != :
#         save_data.append(new_row)

    save_data.append(new_row)

df = pd.DataFrame(save_data)
print(df.shape)

  0%|          | 0/42899811 [00:00<?, ?it/s]

(42899811, 7)


In [21]:
df.head(10)

,src_uuid,src,dst_uuid,dst,relation,dst_type,label
0,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,1987EE68-E918-4765-8082-3DE89890A305,file:\Device\HarddiskVolume2\Users\admin\AppDa...,EVENT_OPEN,file,benign
1,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,1987EE68-E918-4765-8082-3DE89890A305,file:\Device\HarddiskVolume2\Users\admin\AppDa...,EVENT_OPEN,file,benign
2,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_OPEN,file,benign
3,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_OPEN,file,benign
4,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_OPEN,file,benign
5,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_CLOSE,file,benign
6,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_CLOSE,file,benign
7,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,A6BD7B35-CACA-4557-BE82-33B752E35C00,file:Device\HarddiskVolume2\Users\admin\AppDat...,EVENT_CLOSE,file,benign
8,966D983C-607E-4039-BEEF-5E76BC0A4DA8,process:9432&None,F17B89A1-F883-41C2-B0FD-F4FD5B7C21A2,file:\Device\HarddiskVolume2\Users\admin\AppDa...,EVENT_CREATE_OBJECT,file,benign
9,509DFF9D-9792-49A5-8DCC-F60F29425F5E,process:6128&None,084E9EFD-62AA-4FE4-B9DC-118DC1D0222F,file:\Device\HarddiskVolume2,EVENT_FCNTL,file,benign


## entity2id.txt, relation2id, entity2id_objType.txt, relation2id_type.txt

In [22]:
entity_set = set(df['src']).union(set(df['dst']))
relation_set = set(df['relation'])
print(len(entity_set))
print(len(relation_set))

500281
25


In [23]:
def save_entity2id(entity_set):
    entity_idx_dict = dict()
    msg = f'{len(entity_set)}\n'
    msg2 = f'{len(entity_set)}\n'
    obj_types = set()
    for idx, entity in enumerate(entity_set):
        tmp = entity.split(':')
        obj_type = tmp[0]
        entity = ':'.join(tmp[1:])
        obj_types.add(obj_type)
        entity = str(entity).replace(' ', '_')
#         if entity in entity_idx_dict:
#             print('new:', idx, entity, obj_type)
#             print('existed:', entity_idx_dict[entity], entity_obj_dict[entity])
        if entity in entity_idx_dict:
            print('new:', idx, entity, obj_type)
            print('existed:', entity_idx_dict[entity])
        msg += f'{entity}\t{idx}\n'
        msg2 += f'{obj_type}\t{idx}\n'
        entity_idx_dict[entity] = idx
#         entity_obj_dict[entity] = obj_type
    print(obj_types)

    with open(f'{OUTPUT_PATH}/entity2id.txt','w') as f:
        f.write(msg.strip())
    with open(f'{OUTPUT_PATH}/entity2id_objType.txt','w') as f:
        f.write(msg2.strip())
    print(f'Saved {OUTPUT_PATH}/entity2id.txt')
    print(f'Saved {OUTPUT_PATH}/entity2id_objType.txt')
    
    return entity_idx_dict


def save_relation2id(relation_set):
    relation_idx_dict = dict()
    relation_types = set()
    with open(f'{OUTPUT_PATH}/relation2id.txt', 'w') as f, \
         open(f'{OUTPUT_PATH}/relation2id_type.txt', 'w') as f_type:
        
        msg = f'{len(relation_set)}\n'
        msg_type = f'{len(relation_set)}\n'

        for idx, relation in enumerate(relation_set):
            tmp = relation.split(':')
            rel_type = tmp[0]
            relation = ':'.join(tmp[1:])
            relation_types.add(rel_type)

            relation = str(relation).replace(' ', '_')
            msg += f'{relation}\t{idx}\n'
            msg_type += f'{rel_type}\t{idx}\n'
            relation_idx_dict[relation] = idx

        f.write(msg.strip())
        f_type.write(msg_type.strip())

    print(f'Saved {OUTPUT_PATH}/relation2id.txt')
    print(f'Saved {OUTPUT_PATH}/relation2id_type.txt')
    return relation_idx_dict


# def save_relation2id(relation_set):
#     relation_idx_dict = dict()
#     with open(f'{OUTPUT_PATH}/relation2id.txt','w') as f:
#         msg = f'{len(relation_set)}\n'
#         for idx, relation in enumerate(relation_set):
#             relation = str(relation).replace(' ', '_')
#             msg += f'{relation}\t{idx}\n'
#             relation_idx_dict[relation] = idx
#         f.write(msg.strip())
#     print(f'Saved {OUTPUT_PATH}/relation2id.txt')
#     return relation_idx_dict

entity_id_dict = save_entity2id(entity_set)
rel_id_dict = save_relation2id(relation_set)

{'network', 'registry', 'process', 'file'}
Saved ./data/3_openKE/synthesize/entity2id.txt
Saved ./data/3_openKE/synthesize/entity2id_objType.txt
Saved ./data/3_openKE/synthesize/relation2id.txt
Saved ./data/3_openKE/synthesize/relation2id_type.txt


In [24]:
print(len(entity_id_dict))

500281


## train2id.txt, valid2id.txt, test2id.txt, train2id_labels.txt, valid2id_labels.txt, test2id_labels.txt

In [ ]:
triplets = list(zip(df['src'],df['dst'],df['relation'],df['dst_type'],df['label']))
print(len(triplets))
for t in triplets[:30]:
    print(t[4], end=',')

In [ ]:
from collections import Counter
Counter(df['label']).most_common()

In [ ]:
new_triplets = list()
for triplet in tqdm(triplets):
    src_entity = str(':'.join(triplet[0].split(':')[1:])).replace(' ', '_')
    dst_entity = str(':'.join(triplet[1].split(':')[1:])).replace(' ', '_')
    new_triplets.append((
        entity_id_dict[src_entity],
        entity_id_dict[dst_entity],
        rel_id_dict[triplet[2].replace(' ', '_')],
        triplet[3],
        triplet[4]
    ))
len(new_triplets)

In [ ]:
TRAIN_SIZE = 1
VALID_SIZE = 0.1
TEST_SIZE = 0.1
train_dataset = np.array(new_triplets)
_, tmp = train_test_split(train_dataset, test_size=VALID_SIZE+TEST_SIZE, shuffle=True)
valid_dataset, test_dataset = train_test_split(tmp, test_size=TEST_SIZE/(VALID_SIZE+TEST_SIZE),shuffle=True)
print(f'train_dataset.shape: {train_dataset.shape} ({TRAIN_SIZE})')
print(f'valid_dataset.shape: {valid_dataset.shape} ({VALID_SIZE})')
print(f'test_dataset.shape: {test_dataset.shape} ({TEST_SIZE})')

In [ ]:
def save_2id_txt(data, ent_fname, ent_label_fname):
    msg = f'{len(data)}\n'
    msg2 = f'{len(data)}\n'
    for ids in data:
        msg += f'{ids[0]} {ids[1]} {ids[2]}\n'
        msg2 += f'{ids[3]} {ids[4]}\n'
    with open(f'{OUTPUT_PATH}/{ent_fname}', 'w') as f:
        f.write(msg.strip())
    print(f'Saved {OUTPUT_PATH}/{ent_fname}')
    with open(f'{OUTPUT_PATH}/{ent_label_fname}', 'w') as f:
        f.write(msg2.strip())
    print(f'Saved {OUTPUT_PATH}/{ent_label_fname}')

save_2id_txt(train_dataset, 'train2id.txt', 'train2id_label.txt')
save_2id_txt(valid_dataset, 'valid2id.txt', 'valid2id_label.txt')
save_2id_txt(test_dataset, 'test2id.txt', 'test2id_label.txt')